In [5]:
import pickle
import pandas as pd

In [48]:
data_set2 = pd.read_csv('dataset2_v2.csv', sep=',', index_col=0)
category_dict = pd.read_csv('category_dict.csv')

data_set2.insert(1, 'PRODUCT_NAME', data_set2.NAME_x)
del data_set2['NAME_x']
data_set2.insert(9, 'NAME', data_set2.NAME_y)
del data_set2['NAME_y']
data_set2 = pd.merge(data_set2, category_dict, how='left')
data_set2.insert(0, 'NAME_WITH_CODE', data_set2.NAME + data_set2.CODE.apply(lambda x: ' [code:' + str(x) + ']'))
data_set2.insert(0, 'NAME_WITH_CODE_WITH_VALUE', data_set2.NAME_WITH_CODE + data_set2.VALUE.apply(lambda x: '=' + str(x)))

In [131]:
alg_data_dict = pickle.load(open('alg_data_dict_2.pkl', 'r'))
cat_alg_data_dict = pickle.load(open('cat_alg_data_dict_2.pkl', 'r'))

In [132]:
print alg_data_dict.keys()
print cat_alg_data_dict.keys()

['TV', 'IRONS', 'PHONE', 'NOTEBOOK', 'WASHING MACHINE']
['LED TV 43\\"-50\\"    ', 'IRONS               ', 'APPLE IPHONE        ', 'OLED-TV 51\\"-55\\"   ', 'SAMSUNG SMARTPHONES ', 'SMARTPHONES         ', 'LED TV 32\\"-38\\"    ', 'WASHING MACHINES    ', 'LED TV 39\\"-42\\"    ', 'LED TV 15\\"-30\\"    ', 'OLED-TV 56\\"-65\\"   ', 'LED TV 66\\"+        ', 'NOTEBOOK APPLE      ', 'LED TV 51\\"-55\\"    ', 'MINIWASHING MACHINES', 'LED TV 56\\"-65\\"    ', 'NOTEBOOK            ']


In [134]:
def top_k_features(product_id, alg_data_dict, cat_alg_data_dict, k=5):
    product_name = data_set2[data_set2.PRODUCT==product_id]['PRODUCT_NAME'].values[0]
    cat_name = data_set2[data_set2.PRODUCT==product_id]['CATEGORY_NAME'].values[0]
    par_cat_name = data_set2[data_set2.PRODUCT==product_id]['PARENT_CATEGORY_NAME'].values[0]
    print u'ID товара:', product_id
    print u'Товар:', product_name
    print u'Категория:', cat_name
    print u'Надкатегория:', par_cat_name
    par_cat_data = pd.merge(data_set2[data_set2.PRODUCT==product_id], alg_data_dict[par_cat_name], how='inner',\
                     left_on='NAME_WITH_CODE_WITH_VALUE', right_on='NAME=VALUE').\
                sort_values(by='feature_value_weight', ascending=False)\
                [['GROUP_NAME', 'NAME_x', 'VALUE', 'MEASURE', 'feature_value_weight']].drop_duplicates()
    par_cat_data.columns = ['GROUP_NAME', 'NAME_x', 'VALUE', 'MEASURE', 'feature_value_weight']
    try:
        cat_data = pd.merge(data_set2[data_set2.PRODUCT==product_id], cat_alg_data_dict[cat_name], how='inner',\
                         left_on='NAME_WITH_CODE_WITH_VALUE', right_on='NAME=VALUE').\
                    sort_values(by='feature_value_weight', ascending=False)\
                    [['GROUP_NAME', 'NAME_x', 'VALUE', 'MEASURE', 'feature_value_weight']].drop_duplicates()
        par_cat_data.insert(4, 'log_rank', 2 ** (1 - par_cat_data['feature_value_weight'].rank(ascending=False)))
        cat_data.insert(4, 'log_rank', 2 ** (1 - cat_data['feature_value_weight'].rank(ascending=False)))
        total_data = pd.merge(par_cat_data, cat_data, how='outer',\
                              on=['GROUP_NAME', 'NAME_x', 'VALUE', 'MEASURE']).fillna(0)
        total_data.insert(0, 'total_rank', total_data.log_rank_x + total_data.log_rank_y)
        total_data.sort_values(by='total_rank', ascending=False, inplace=True)
        return total_data.head(5)[['GROUP_NAME', 'NAME_x', 'VALUE', 'MEASURE']]
    except:
        return par_cat_data.head(5)[['GROUP_NAME', 'NAME_x', 'VALUE', 'MEASURE']]
    
        

In [138]:
a = top_k_features(20023532, alg_data_dict, cat_alg_data_dict)
a

ID товара: 20023532
Товар: Утюг Scarlett SC-334S                                                                                        
Категория: IRONS               
Надкатегория: IRONS


,GROUP_NAME,NAME_x,VALUE,MEASURE
3,Цвет и вес,Вес,1,кг
0,Защита от накипи,Самоочистка от накипи,Да,0
1,Серия модели,Серия,Ted,0
5,Система увлажнения,Паровой удар,120,г
2,0,Гарантия,1 год,0


In [139]:
a = top_k_features(10008191, alg_data_dict, cat_alg_data_dict)
a

ID товара: 10008191
Товар: Телевизор Sony KDL55W808C                                                                                    
Категория: LED TV 51\"-55\"    
Надкатегория: TV


,GROUP_NAME,NAME_x,VALUE,MEASURE
0,Подключение к сети LAN/Internet,Поддержка Wi-Fi,через встроенный модуль,0
9,"Цвет, размеры и вес",Вес,18,кг
3,Системы обработки изображения,Технология,1000 Гц,0
1,Дистанционное управление,Тип дистанционного управления,ИК,0
4,0,Ширина,123,см


In [140]:
data_set2.CATEGORY_NAME.unique()

array(['LED TV 32\\"-38\\"    ', 'LED TV 39\\"-42\\"    ',
       'LED TV 51\\"-55\\"    ', 'LED TV 15\\"-30\\"    ',
       'LED TV 43\\"-50\\"    ', 'LED TV 56\\"-65\\"    ',
       'LED TV 66\\"+        ', 'OLED-TV 51\\"-55\\"   ',
       'OLED-TV 56\\"-65\\"   ', 'WASHING MACHINES    ',
       'IRONS               ', 'MINIWASHING MACHINES',
       'NOTEBOOK            ', 'SMARTPHONES         ',
       'VAIO NOTEBOOKS      ', 'APPLE IPHONE        ',
       'NOTEBOOK APPLE      ', 'SAMSUNG SMARTPHONES '], dtype=object)

In [142]:
data_set2[data_set2.CATEGORY_NAME=='VAIO NOTEBOOKS      '].PRODUCT.unique()[:10]

array([30005105, 30005107, 30005150, 30005151, 30005161, 30005162,
       30005207, 30005208, 30005209, 30005215])

In [145]:
data_set2.PRODUCT.sample(10)

278848    30019363
274752    30019003
321962    30021225
272017    30018859
3329      10000473
364747    30023641
261026    30018097
213568    30015125
431346    30027141
389214    30025166
Name: PRODUCT, dtype: int64

In [148]:
for product_id in data_set2.PRODUCT.sample(10):
    a = top_k_features(product_id, alg_data_dict, cat_alg_data_dict)
    print a
    print '\n'

ID товара: 11023825
Товар: Телевизор Samsung LE-22 B650 T6W                                                                             
Категория: LED TV 15\"-30\"    
Надкатегория: TV
   GROUP_NAME             NAME_x                     VALUE MEASURE
0           0             Ширина                        54      см
6       Экран   Диагональ экрана              22"(55.8 см)       0
1       Экран  Разрешение экрана  1366x768 Пикс (HD Ready)       0
7  Интерфейсы          Вход HDMI                         1      шт
2  Интерфейсы     Вход RCA аудио                         2       0


ID товара: 30016819
Товар: Смартфон Apple iPhone 5 32Gb White (MD300RU/A)                                                               
Категория: APPLE IPHONE        
Надкатегория: PHONE
            GROUP_NAME                       NAME_x              VALUE MEASURE
0      Стандарты связи             Поддержка 4G LTE                 Да       0
6               Модель                       Модель            